# Cumulative event counter

### This notebook counts the cumulative spatio-temporal distribution of cellular events in the local neighbourhood of a series of apoptosis events

1. Load dictionary of ~apoptoses~ events and their corresponding times to base the spatiotemporal scans around 
2. Load corresponding tracking data and specify scan parameters
3. Iteratively calculate spatiotemporal distribution of cellular events around apoptoses, saving each distribution out as a .npy distance-time heatmap
4. Compile list of spatiotemporal heatmaps as cumulative heatmap
5. Misc. section: comparing different experimental conditions

In [3]:
import calculate_radial_analysis as calculate
import tools
import render
import dataio

import json
import glob
import os
import re
import numpy as np
from natsort import natsorted
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path

## 1) Event dictionary loader

Specify cell type to filter out unwanted apoptoses, want end result of event_dict that takes the format of :`event_dict = {event_ID:event_time}` 

where `event_ID` takes the format of: `GV{experiment_ID}_Pos{position_ID}_{cell_ID}_{cell_type}` 

For example: 

```
event_dict = {'GV0794_Pos4_393_GFP': '766',
             'GV0794_Pos7_364_GFP': '550',
             'GV0794_Pos7_457_GFP': '805',
             'GV0794_Pos7_369_GFP': '833',
             'GV0794_Pos7_292_GFP': '884',
             'GV0795_Pos4_358_GFP': '628',
             'GV0795_Pos4_18_GFP': '855',
             'GV0795_Pos4_1356_GFP': '992',
             'GV0795_Pos4_359_GFP': '1133',
             'GV0796_Pos4_542_GFP': '581',
             'GV0796_Pos4_1334_GFP': '719',
             'GV0796_Pos4_1586_GFP': '785'}
 ```
 
Function to apoptosis event dictionary from preexisting data store:

`dataio.apoptosis_list_loader(path_to_apop_lists, cell_type)`

Where the `path_to_apop_lists` is the directory containing the apoptosis information and the `cell_type` is either `'Scr'` or `'WT'`.

In [7]:
path_to_apop_lists = '/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/final/all'
event_dict = dataio.apoptosis_list_loader(path_to_apop_lists, 'Scr')
event_dict

{'GV0797_Pos7_14_RFP': '99',
 'GV0797_Pos7_45_RFP': '201',
 'GV0797_Pos7_5_RFP': '254',
 'GV0797_Pos7_146_RFP': '388',
 'GV0797_Pos7_145_RFP': '398',
 'GV0797_Pos7_30_RFP': '393',
 'GV0797_Pos7_106_RFP': '424',
 'GV0797_Pos7_9_RFP': '405',
 'GV0797_Pos7_204_RFP': '487',
 'GV0797_Pos7_105_RFP': '473',
 'GV0797_Pos7_200_RFP': '486',
 'GV0797_Pos7_199_RFP': '625',
 'GV0797_Pos7_208_RFP': '475',
 'GV0797_Pos7_207_RFP': '475',
 'GV0797_Pos7_203_RFP': '516',
 'GV0797_Pos23_31_RFP': '166',
 'GV0797_Pos23_26_RFP': '60',
 'GV0797_Pos23_260_RFP': '235',
 'GV0797_Pos23_125_RFP': '79',
 'GV0797_Pos23_336_RFP': '456',
 'GV0797_Pos23_252_RFP': '207',
 'GV0797_Pos23_301_RFP': '207',
 'GV0797_Pos23_36_RFP': '180',
 'GV0797_Pos23_319_RFP': '556',
 'GV0797_Pos23_556_RFP': '638',
 'GV0798_Pos22_34_RFP': '54',
 'GV0798_Pos22_36_RFP': '125',
 'GV0798_Pos22_37_RFP': '469',
 'GV0798_Pos22_358_RFP': '461',
 'GV0798_Pos22_26_RFP': '761',
 'GV0798_Pos22_696_RFP': '871',
 'GV0798_Pos22_9_RFP': '190',
 'GV0798_Po

### Update: 

Currently implementing focal scan around divisions (originally was just apoptoses)

In [2]:
with open('/home/nathan/data/kraken/h2b/giulia/experiment_information/divisions/scr_div_dict_confident.json', 'r') as file:
    event_dict = json.load(file)
event_dict

{'GV0794_Pos5_15_RFP': 323,
 'GV0794_Pos5_6_RFP': 132,
 'GV0794_Pos5_41_RFP': 112,
 'GV0794_Pos5_7_RFP': 505,
 'GV0794_Pos6_52_RFP': 57,
 'GV0794_Pos7_60_RFP': 548,
 'GV0794_Pos7_300_RFP': 498,
 'GV0794_Pos7_58_RFP': 301,
 'GV0794_Pos7_294_RFP': 449,
 'GV0794_Pos7_56_RFP': 378,
 'GV0794_Pos7_293_RFP': 510,
 'GV0795_Pos3_46_RFP': 535,
 'GV0795_Pos4_36_RFP': 99,
 'GV0795_Pos4_30_RFP': 407,
 'GV0795_Pos4_12_RFP': 67,
 'GV0795_Pos4_19_RFP': 394,
 'GV0795_Pos4_24_RFP': 92,
 'GV0795_Pos5_164_RFP': 285,
 'GV0795_Pos5_14_RFP': 507,
 'GV0795_Pos5_376_RFP': 472,
 'GV0795_Pos5_50_RFP': 476,
 'GV0795_Pos5_52_RFP': 395,
 'GV0795_Pos5_37_RFP': 351,
 'GV0795_Pos5_19_RFP': 120,
 'GV0795_Pos5_169_RFP': 370,
 'GV0795_Pos5_6_RFP': 449,
 'GV0795_Pos5_29_RFP': 775,
 'GV0795_Pos5_13_RFP': 438,
 'GV0795_Pos5_39_RFP': 363,
 'GV0795_Pos6_150_RFP': 572,
 'GV0795_Pos6_16_RFP': 46,
 'GV0795_Pos6_35_RFP': 416,
 'GV0795_Pos6_151_RFP': 284,
 'GV0795_Pos6_164_RFP': 442,
 'GV0795_Pos6_377_RFP': 414,
 'GV0795_Pos6_444_

In [3]:
### optional parallelisation step
N_focal = len(event_dict)
para_frac_min, para_frac_max = int(0*N_focal), int(1*N_focal)  ## ie only do first 1/4 of all apops
delete_list =[]
for i, j in enumerate(event_dict):
    if not para_frac_min <= i< para_frac_max: 
        delete_list.append(j)
for i in delete_list:
    del event_dict[i]
print("Length of modified dict vs original:", len(event_dict), "/", N_focal)

Length of modified dict vs original: 748 / 748


## 2) Load corresponding cell tracking data and specifiy other parameters

`dataio.hdf5_file_finder(parent_folder_hdf5)` loads a list of all hdf5 files when supplied with the parent directory of all the experiments. This relies on a typical sub-parent directory structure of `'GV****/Pos*/*aligned/HDF/segmented.hdf5'` (aligned folder optional)

In [4]:
### load tracking data
parent_folder_hdf5 = '/home/nathan/data/kraken/h2b/giulia'
hdf5_file_list = dataio.hdf5_file_finder(parent_folder_hdf5)

### specify output path and neighbourhood scan parameters
radius, t_range, num_bins = 1600, 1600, 40#800, 800, 20#400, 400, 10
output_path = '/home/nathan/data/results/radial_analysis_output/follow/scr_div_wt_div/individual_spatiotemporal_heatmaps/{}.{}.{}/scr_apop_wt_div'.format(radius, t_range, num_bins)
Path(output_path).mkdir(parents=True, exist_ok=True)

## 3) Iteratively calculate and save out individual heatmaps

This step can be time consuming (sequential scans around 385 apoptoses took 7 hours on i7)

Function to calculate:

`calculate.iterative_heatmap_generator(subject_cells = 'WT', subject_event = 'DIVIDE', apop_dict, hdf5_file_list, radius, t_range, num_bins, output_path)`

Where the necessary input variables are as follows:
- `subject_cells` = The population of cells to scan over looking for events, 'WT'/'Scr'/'All'
- `subject_event` = The event type to look for, 'DIVIDE' or 'APOPTOSIS'
- `apop_dict` = Previously defined python dictionary of {apoptosis ID: apoptosis time}
- `hdf5_file_list` = List of locations of tracking files
- `radius` = Distance to scan over around apoptosis
- `t_range` = Time range to scan over around apoptosis (will scan t_range/2 before and after apoptosis)
- `num_bins` = Number of bins the histogram is sorted into
- `output_path` = Empty directory to save out individual apoptosis heatmap .npy arrays into


In [ ]:
N_cells, error_log, success_log = calculate.iterative_heatmap_generator(
    'WT', ## 'Scr',
    'DIVIDE', 
    event_dict, 
    hdf5_file_list, 
    radius, 
    t_range, 
    num_bins, 
    output_path)

If you want to save out raw list of cell IDs, distance and frames, enter 'y', else just press entery


  0%|          | 0/748 [00:00<?, ?it/s][INFO][2021/09/02 06:23:42 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 06:23:42 PM] Loading tracks/obj_type_1


ID GV0794_Pos5_15_RFP
Loading GV0794/Pos5/
Btrack version no.: 0.4.2


[INFO][2021/09/02 06:23:42 PM] Loading objects/obj_type_1 (31315, 5) (20377 filtered: area>=100)
[INFO][2021/09/02 06:23:42 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos5/Pos5_aligned/HDF/segmented.hdf5
[INFO][2021/09/02 06:23:42 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2021/09/02 06:23:42 PM] Loading tracks/obj_type_2
[INFO][2021/09/02 06:23:43 PM] Loading objects/obj_type_2 (61992, 5) (46754 filtered: area>=100)
[INFO][2021/09/02 06:23:43 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0794/Pos5/Pos5_aligned/HDF/segmented.hdf5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0794/Pos5/


...

ID GV0794_Pos5_6_RFP


  0%|          | 2/748 [00:21<2:03:36,  9.94s/it]

ID GV0794_Pos5_41_RFP


  0%|          | 3/748 [00:27<1:42:55,  8.29s/it]

ID GV0794_Pos5_7_RFP


In [ ]:
N_cells ## scope of analysis (number of apoptoses scanned around)

In [ ]:
len(error_log)

In [ ]:
error_log, success_log

In [ ]:
stop

## 4) Compile individual heatmaps into cumulative plots 

Compile cumulative heatmap by iteratively adding single-apoptosis heatmaps from previous output or different pre-established output 

In [ ]:
single_cell_heatmap_location = output_path
#single_cell_heatmap_location = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/individual_spatiotemporal_heatmaps/400.400.10/scr_apop_wt_div'
save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/wt_apop_wt_div'
Path(save_path).mkdir(parents=True, exist_ok=True)
radius, t_range, num_bins = 400, 400, 10

In [ ]:
N_cells, N_events = np.zeros((num_bins, num_bins)), np.zeros((num_bins, num_bins))
N = 0
for heatmap_ID in natsorted(os.listdir(single_cell_heatmap_location)):
    if 'N_cells' in heatmap_ID:
        N_cells += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
    if 'N_events' in heatmap_ID:
        N_events += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
        N+=1
N

In [ ]:
print("The number of WT. cells counted is equal to = ", N_cells.sum())

Function to plot heatmaps: 

`plot_cumulative(input_2d_hist, num_bins, radius, t_range, title, expt_label, cb_label, save_path, SI = True)`

Where the variables needed are as follows:

- `input_2d_hist` = Input 2D histogram
- `num_bins` = Number of bins the histogram is sorted into
- `radius` = Spatial extent of scan
- `t_range` = Temporal range of scan
- `title` = Title/save out filename of heatmap
- `label` = Optional label for heatmap
- `cb_label` = Label for colourbar
- `save_path` = Ouput save path, if empty string then plot will not be saved out
- `SI` = If True then plots the xy in SI base unit instead of pixel/frames

In [ ]:
save_path

In [ ]:
render.plot_cumulative(input_2d_hist = N_cells,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of wild-type cells \n around wild-type apoptoses (N={})'.format(N), 
                label = '', 
                cb_label = 'Number of wild-type cell apperances', 
                save_path = save_path,#'/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div',
                SI = True) ## SI is standard 

In [ ]:
render.plot_cumulative(input_2d_hist = N_events,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of wild-type mitoses \n around wild-type apoptoses (N={})'.format(N), 
                label = '', 
                cb_label = 'Number of wild-type divisions', 
                save_path = save_path,#'/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/400.400.10/scr_apop_wt_div',
                SI = True)

In [ ]:
render.plot_cumulative(input_2d_hist = N_events/N_cells,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of probability of wild-type division \n around wild-type apoptoses (N={})'.format(N), 
                label = '', 
                cb_label = 'Probability of wild-type division', 
                save_path = save_path,#'/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/cumulative_plots/400.400.10/scr_apop_wt_div',
                SI = True)

# 5) Miscellaneous section: segregating cumulative plots based on experiment type

50:50 expt

In [ ]:
single_cell_heatmap_location = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/individual_spatiotemporal_heatmaps/400.400.10/scr_apop_wt_div'
save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types'
Path(save_path).mkdir(parents=True, exist_ok=True)

In [ ]:
expt_list_5050 = ['GV0794/Pos4',
                 'GV0794/Pos7',
                 'GV0794/Pos16',
                 'GV0795/Pos4',
                 'GV0795/Pos7',
                 'GV0795/Pos17',
                 'GV0796/Pos4',
                 'GV0796/Pos7',
                 'GV0796/Pos21',
                 'GV0796/Pos22',
                 'GV0797/Pos4',
                 'GV0797/Pos7',
                 'GV0797/Pos20',
                 'GV0797/Pos21',
                 'GV0798/Pos4',
                 'GV0798/Pos7',
                 'GV0798/Pos20',
                 'GV0798/Pos21',
                 'GV0800/Pos4', 
                 'GV0800/Pos7',
                 'GV0800/Pos16',
                 'GV0801/Pos4',
                 'GV0802/Pos4',
                 'GV0804/Pos4',
                 'GV0805/Pos4',
                 'GV0806/Pos4',
                 'GV0807/Pos4',
                 'GV0812/Pos0',
                 'GV0812/Pos1',
                 'GV0812/Pos2',
                 'GV0812/Pos3',
                 'GV0812/Pos4',
                 'GV0812/Pos5',
                 'GV0813/Pos0',
                 'GV0813/Pos1',
                 'GV0813/Pos2',
                 'GV0813/Pos3',
                 'GV0813/Pos4',
                 'GV0813/Pos5',
                 'GV0814/Pos0',
                 'GV0814/Pos11',
                 'GV0814/Pos12',
                 'GV0814/Pos23',
                 'GV0815/Pos0',
                 'GV0815/Pos11',
                 'GV0815/Pos12',
                 'GV0815/Pos23',
                 'GV0816/Pos0',
                 'GV0816/Pos13',
                 'GV0816/Pos14',
                 'GV0816/Pos23',
                 'GV0817/Pos0',
                 'GV0817/Pos11',
                 'GV0817/Pos12',
                 'GV0817/Pos23',
                 'GV0818/Pos12',
                 'GV0818/Pos13',
                 'GV0819/Pos0',
                 'GV0819/Pos1',
                 'GV0819/Pos2',
                 'GV0819/Pos3',
                 'GV0819/Pos4',
                 'GV0819/Pos5',
                 'GV0820/Pos0',
                 'GV0820/Pos1',
                 'GV0820/Pos2',
                 'GV0820/Pos3',
                 'GV0820/Pos4',
                 'GV0820/Pos5']

In [ ]:
N_cells, N_events = np.zeros((num_bins, num_bins)), np.zeros((num_bins, num_bins))
N = 0
for heatmap_ID in natsorted(os.listdir(single_cell_heatmap_location)):
    #print(heatmap_ID.split('_P'))
    expt = 'GV' +str(re.findall(r"GV(\d+)", heatmap_ID)[0])
    position = re.findall(r"Pos(\d+)", heatmap_ID)[0]
    if os.path.join(expt,'Pos'+position) in expt_list_5050:
        if 'N_cells' in heatmap_ID:
            N_cells += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
        if 'N_events' in heatmap_ID:
            N_events += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
            N+=1
N

In [ ]:
render.plot_cumulative(input_2d_hist = N_cells,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of wild-type cells \n around Scribble apoptoses (N={}, 50:50 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Number of cell apperances', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types',
                SI = True) ## SI is standard 

In [ ]:
render.plot_cumulative(input_2d_hist = N_events,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of wild-type division \n around Scribble apoptoses (N={}, 50:50 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Number of wild-type divisions', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types',
                SI = True)

In [ ]:
render.plot_cumulative(input_2d_hist = N_events/N_cells,
                num_bins  = 10,
                radius  = 400,
                t_range = 400,  
                title = 'Spatiotemporal dist. of probability of Scribble division \n around wild-type apoptoses (N={}, 50:50 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Probability of division', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types',
                SI = True)

90:10 wt:scr

In [ ]:
expt_list_9010 = ['GV0794/Pos3',
 'GV0794/Pos8',
 'GV0794/Pos15',
 'GV0795/Pos3',
 'GV0795/Pos8',
 'GV0795/Pos16',
 'GV0796/Pos3',
 'GV0796/Pos8',
 'GV0796/Pos19',
 'GV0796/Pos20',
 'GV0797/Pos3',
 'GV0797/Pos8',
 'GV0797/Pos18',
 'GV0797/Pos19',
 'GV0798/Pos3',
 'GV0798/Pos8',
 'GV0798/Pos18',
 'GV0798/Pos19',
 'GV0800/Pos3',
 'GV0800/Pos8',
 'GV0800/Pos15',
 'GV0801/Pos3',
 'GV0802/Pos3',
 'GV0804/Pos3',
 'GV0805/Pos3',
 'GV0806/Pos3',
 'GV0807/Pos3']

In [ ]:
N_cells, N_events = np.zeros((num_bins, num_bins)), np.zeros((num_bins, num_bins))
N=0
for heatmap_ID in natsorted(os.listdir(single_cell_heatmap_location)):
    #print(heatmap_ID.split('_P'))
    expt = 'GV' +str(re.findall(r"GV(\d+)", heatmap_ID)[0])
    position = re.findall(r"Pos(\d+)", heatmap_ID)[0]
    if os.path.join(expt,'Pos'+position) in expt_list_9010:
        if 'N_cells' in heatmap_ID:
            N_cells += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
        if 'N_events' in heatmap_ID:
            N_events += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
            N+=1
N

In [ ]:
render.plot_cumulative(input_2d_hist = N_cells,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of wild-type cells \n around Scribble apoptoses (N={}, 90:10 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Number of cell apperances', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types',
                SI = True) ## SI is standard 

In [ ]:
render.plot_cumulative(input_2d_hist = N_events,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of wild-type division \n around Scribble apoptoses (N={}, 90:10 wt:scr expts only)'.format(N),
                label = '', 
                cb_label = 'Number of wild-type divisions', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types',
                SI = True)

In [ ]:
render.plot_cumulative(input_2d_hist = N_events/N_cells,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of probability of wild-type division \n around Scribble apoptoses (N={}, 90:10 wt:scr expts only)'.format(N),
                label = '', 
                cb_label = 'Probability of division', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types',
                SI = True)

10:90 wt:scr

In [ ]:
expt_list_1090 = ['GV0794/Pos5',
 'GV0794/Pos6',
 'GV0795/Pos5',
 'GV0795/Pos6',
 'GV0795/Pos18',
 'GV0796/Pos5',
 'GV0796/Pos6',
 'GV0796/Pos23',
 'GV0796/Pos24',
 'GV0797/Pos5',
 'GV0797/Pos6',
 'GV0797/Pos22',
 'GV0797/Pos23',
 'GV0798/Pos5',
 'GV0798/Pos6',
 'GV0798/Pos22',
 'GV0798/Pos23',
 'GV0800/Pos5',
 'GV0800/Pos6',
 'GV0800/Pos17',
 'GV0801/Pos5',
 'GV0802/Pos5',
 'GV0804/Pos5',
 'GV0805/Pos5',
 'GV0806/Pos5',
 'GV0807/Pos5']

In [ ]:
N_cells, N_events = np.zeros((num_bins, num_bins)), np.zeros((num_bins, num_bins))
N=0
for heatmap_ID in natsorted(os.listdir(single_cell_heatmap_location)):
    #print(heatmap_ID.split('_P'))
    expt = 'GV' +str(re.findall(r"GV(\d+)", heatmap_ID)[0])
    position = re.findall(r"Pos(\d+)", heatmap_ID)[0]
    if os.path.join(expt,'Pos'+position) in expt_list_1090:
        if 'N_cells' in heatmap_ID:
            N_cells += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
        if 'N_events' in heatmap_ID:
            N_events += np.load(os.path.join(single_cell_heatmap_location,heatmap_ID))
            N+=1
N

In [ ]:
render.plot_cumulative(input_2d_hist = N_cells,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of wild-type cells around \nScribble apoptoses (N={}, 10:90 wt:scr expts only)'.format(N), 
                label = '', 
                cb_label = 'Number of cell apperances', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types',
                SI = True) ## SI is standard 

In [ ]:
render.plot_cumulative(input_2d_hist = N_events,
                num_bins  = 10,
                radius  = 400,
                t_range = 400,  
                title = 'Spatiotemporal dist. of wild-type division around \nScribble apoptoses (N={}, 10:90 wt:scr expts only)'.format(N),
                label = '', 
                cb_label = 'Number of wild-type divisions', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types',
                SI = True)

In [ ]:
render.plot_cumulative(input_2d_hist = N_events/N_cells,
                num_bins  = 10,
                radius  = 400,
                t_range = 400, 
                title = 'Spatiotemporal dist. of probability of wild-type division around \nScribble apoptoses (N={}, 10:90 wt:scr expts only)'.format(N),
                label = '', 
                cb_label = 'Probability of division', 
                save_path = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/follow/cumulative_plots/400.400.10/scr_apop_wt_div/expt_types',
                SI = True)

Write a function to do a time evolving plot that iteratively adds single cell heat maps according to the apoptosis time?